## Chapter 16. Operator Overloading

### Overloaded Signatures

In [1]:
# Example 16-1. vector_v6.py: unary operators - and + added to Example 12-16

from array import array
import reprlib
import math
import functools
import operator
import itertools


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

# Example 16-1. vector_v6.py: unary operators - and + added to Example 12-16
    def __abs__(self):
        return math.hypot(*self)

    def __neg__(self):
        return Vector(-x for x in self)  # <1>

    def __pos__(self):
        return Vector(self)  # <2>
# end:: 16-1

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    __match_args__ = ('x', 'y', 'z', 't')

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

# Example 16-10. vector_v6.py: operator + methods added to vector_v5.py
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other

In [2]:
v1 = Vector([3, 4])
print(abs(v1))
print(-v1)
print(+v1)

5.0
(-3.0, -4.0)
(3.0, 4.0)


In [3]:
# Example 16-2. A change in the arithmetic context precision may cause x to differ from +x

import decimal
ctx = decimal.getcontext()  # <1>
ctx.prec = 40  # <2>
one_third = decimal.Decimal('1') / decimal.Decimal('3')  # <3>
print(one_third)  # <4>
print(one_third == +one_third)  # <5>

ctx.prec = 28  # <6>
print(one_third == +one_third)  # <7>
print(+one_third)  # <8>

0.3333333333333333333333333333333333333333
True
False
0.3333333333333333333333333333


In [4]:
# Example 16-3. Unary + produces a new Counter without zeroed or negative tallies

from collections import Counter

ct = Counter('abracadabra')
print(ct)

ct['r'] = -3
ct['d'] = 0
print(ct)
print(+ct)  # discard a negative and zero

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})
Counter({'a': 5, 'b': 2, 'c': 1, 'd': 0, 'r': -3})
Counter({'a': 5, 'b': 2, 'c': 1})


### Overloading + for Vector Addtion

In [7]:
v1 = Vector([3, 4, 5])
v2 = Vector([6, 7, 8])
print(v1 + v2)
print(v1 + v2 == Vector([3 + 6, 4 + 7, 5 + 8]))

(9.0, 11.0, 13.0)
True


In [8]:
v1 = Vector([3, 4, 5, 6])
v3 = Vector([1, 2])
v1 + v3

Vector([4.0, 6.0, 5.0, 6.0])

In [9]:
# Example 16-5. Vector.__add__ take #1 supports non-Vector objects, too
v1 = Vector([3, 4, 5])
v1 + (10, 20, 30)

Vector([13.0, 24.0, 35.0])

In [10]:
# Example 16-6. Vector.__add__ take #1 fails with non-Vector left operands
v1 = Vector([3, 4, 5])
(10, 20, 30) + v1

Vector([13.0, 24.0, 35.0])

In [ ]:
# Example 16-8. Vector.__add__ method needs an iterable operand
v1 + 1

TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

In [12]:
# Example 16-9. Vector.__add__ method needs an iterable with numeric items
v1 + 'ABC'

TypeError: unsupported operand type(s) for +: 'Vector' and 'str'